In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from sklearn import preprocessing, linear_model

# Clustering


### Machine Learning and Computational Statistics (DSC6135)

### Instructors: Weiwei Pan (Harvard), Javier Zazo (Harvard), Melanie F. Pradier (Harvard)

* Supervised Learning

* **Unsupervised Learning**

<img src="figs/unsupervised.png" width="80%">

## Some Application Examples

<img src="figs/ap1.png" width="80%">

<img src="figs/ap2.png" width="80%">

<img src="figs/ap3.png" width="80%">

<img src="figs/ap4.png" width="80%">

## How to cluster these points?

<img src="figs/points1.png" width="80%">

## How to cluster these points?

<img src="figs/points2.png" width="80%">

## Key Questions    
    
* How many clusters?

<img src="figs/howmanyclusters.jpg" width="80%">


## Key Questions

* What shape for the clusters?

<img src="figs/shapes1.png" width="60%">
<img src="figs/shapes2.png" width="60%">


# K-Means

* **Input**:
    * $N$ datapoints: $x_1, x_2, \ldots, x_N$
    * parameters $K$ (number of clusters)

* **Goals of K-means**:
    1. Assign each datapoint to one of $K$ clusters<br> (*Assumption*: clusters are exclusive)
    2. Minimize Euclidean distance from datapoints to cluster centers<br>
    (*Assumption*: isotropic Euclidean - all features weighted equally - is a good metric)

### K-means output:

* Centroid vectors (one per cluster $k=1,\ldots,K$)

    $$ \mu_k = [\mu_{k1}, \mu_{k2}, \ldots, \mu_{kD}] $$
    
* Cluster Assignments (one per datapoint $n=1,\ldots,N$)

    $$ z_n = [z_{n1}, z_{n2}, \ldots, z_{nK}] = [0\, 1\, 0\, 0\,  0] $$


### K-means Optimization problem

$$\min_{z,\mu} \sum^N_{n=1} \sum^K_{k=1} z_{nk} \, \mathrm{distance}(x_n,\mu_k)$$

where

$$\mathrm{distance}(x_n,\mu_k) = ||x_n - \mu_k ||^2$$

Directly optimizing this expression is problematic!!

### Iterative Algorithm (Expectation-Maximization)

* Initialize cluster means randomly

* Repeat until converged

<img src="figs/alg1.png" width="80%">

### Iterative Algorithm (Expectation-Maximization)

* Initialize cluster means randomly

* Repeat until converged

<img src="figs/alg2.png" width="80%">

Demo!

https://www.naftaliharris.com/blog/visualizing-k-means-clustering/

* For the same data and number of clusters, can you find different clustering configurations?

* What does this mean about the objective function?

A different demo:

http://stanford.edu/class/ee103/visualizations/kmeans/kmeans.html

## Interesting Notes

* K-means boundaries are linear

<img src="figs/kmeans.png" width="80%">

* Each step in EM-algorithm leads to equal or lower cost than previous one

* Initialization important (leads to different solutions)
* Use cost to decide among multiple runs of $K$-means

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

In [ ]:
# K-means algorithm from scratch (doing Expectation-Maximization)
from sklearn.metrics import pairwise_distances_argmin

def find_clusters(X, n_clusters, rseed=2):
    # 1. Randomly choose clusters
    rng = np.random.RandomState(rseed)
    i = rng.permutation(X.shape[0])[:n_clusters]
    centers = X[i]
    
    while True:
        # 2a. Get cluster assignments based on closest center
        assignments = pairwise_distances_argmin(X, centers)
        
        # 2b. Find new centers from means of points
        new_centers = np.array([X[assignments == i].mean(0)
                                for i in range(n_clusters)])
        
        # 2c. Check for convergence
        if np.all(centers == new_centers):
            break
        centers = new_centers
    
    return centers, assignments

## How to initialize the centroids

* Step 1: choose an example uniformly at random as first centroid
* Repeat for k = 2, 3, … K: 
    - Choose example based on distance from nearest centroid:
        $$
        P(\mu_k) \propto \min_{1,2,\ldots,k-1} \text{dist}(x_n,\mu_j)
        $$

* This procedure initializes clusters far from each other

* There are some theoretical guarantees of worst-case performance...
* ... but the problem is hard!

How do we address this difficulties?

* Multiple initializations --> choose based on some score evaluation / metrics

* There are no guarantees that your cluster results will be good with a single one!

## Multiple initializations: scoring

<img src="figs/cluster_init.png" width="80%">



## How do we evaluate the score?

* Many possibile metrics: based on true labels, distance considerations between clusters...
* In practical, we use shiloutte coefficient:

<img src="figs/shilhouette_score.png" width="80%">


## How to choose number of clusters $K$?

<img src="figs/varyingK.png" width="80%">



### Can we use the cost function?

<img src="figs/elbow0.jpg" width="70%">





* We should not!! Instead, regularization

$$\textrm{loss function} = \textrm{cost} + \lambda \, \textrm{penalty}(K)$$

* Or... based on elbow:

<img src="figs/elbow.jpg" width="70%">


## Disadvantages of $K$-means

* Hard assignments: clusters are exclusive


* Weight equally each feature (isotropic Euclidean distance)

## Gaussian Mixture Models

### Improving $K$-means:

1. Assign each datapoint to one of $K$ clusters<br> (*Assumption*: clusters are exclusive)<br>
    **Improvement**: soft-probabilistic assignments<br>
    
    
2. Minimize Euclidean distance from datapoints to cluster centers<br>
    (*Assumption*: isotropic Euclidean - all features weighted equally - is a good metric)<br>
    **Improvement**: model cluster covariance

```
gmm = GMM(n_components=4, covariance_type='full')
```
<img src="figs/GMM.png?1" width="80%">

## Multivariate Gaussian

<img src="figs/multivariate_gaussian.png?1" width="70%">

* Probability density function:
$$f_Y(x)=\frac{1}{\sqrt{(2\pi)^n|\boldsymbol\Sigma|}}
\exp\left(-\frac{1}{2}({x}-{\mu})^T{\boldsymbol\Sigma}^{-1}({x}-{\mu})
\right)$$

* Mean: $\mu \in\mathbb{R}^F$
* Covariance: $\Sigma \in\mathbb{R}^{F\times F}$


## Covariance models

<img src="figs/cov.png?1" width="80%">


## Parameters for Gaussian Mixture Models:

* mean vectors --> one per cluster $k$.
$$\mu_k=[\mu_{k1},\mu_{k2},\ldots,\mu_{kF}]$$

* covariance matrix --> one per cluster $k$.
$$ \Sigma_k \in \mathbb{R}^{F\times F}, \qquad\forall k\in \{1,\ldots,K\} $$

* soft assignments --> one per example $n$ in $1,\ldots, N$
$$ r_n = [r_{n1}, r_{n2},\ldots, r_{nK}] \qquad\forall n\in \{1,\ldots,N\} $$

TALK about probabilistic perspective

### Multivariate Gaussian parameter estimation Training

* How to fit the parameters of a multivariate Gaussian from data?

$$ X = {x_{1}, x_2, \dots, x_N}.$$

* Maybe with a log-maximum likelihood estimate?

* Mean is the average of the samples:

$$ \mu = \frac{1}{N} \sum_{i=1}^N x_i$$

* The covariance matrix:
    
$$\Sigma = \frac{1}{N} \sum_{i=1}^N (x_i - \mu) (x_i - \mu)^T$$

## Gaussian Mixture Model training

* We can estimate the parameters of multivariate Gaussians easily...

* ... but we don't know to which cluster each data point belongs to.


* We can, once more, maximize log-likelihood!

$$\max_{\mu_k, \Sigma_k} \quad \sum_{i=1}^{N}\sum_{k=1}^{K} \log p(x_i, z_{ik} =1 | \mu_k, \Sigma_k) $$

* How do we optimize?

*  We don't have cluster assignments, and we don't have mean and covariance estimates --> but we will use soft assignments!

## Expectation - Maximization

* We repeat the following procedure until convergence

* On each step, we assume the other variables fixed.

---


1. E-step: Update soft assignments $r$ (and normalize):

$$ r_{ik} \propto p(x_i, z_k =1 | \mu_k, \Sigma_k) \quad \forall k, i$$ 

2. M-step: Update means and covariances:

$$ \mu = \frac{1}{N} \sum_{i=1}^N r_{ik} x_i$$

$$\Sigma = \frac{1}{N} \sum_{i=1}^N  r_{ik} (x_i - \mu) (x_i - \mu)^T$$


## $K$-means is a GMM with:

* hard assignments --> probabilities $r$ are one-hot encodings.
* spherical covariances

$$ \Sigma_k = \lim_{\sigma\rightarrow 0}\sigma^2 \left[\begin{array}{ccc}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{array} \right]$$